<a href="https://colab.research.google.com/github/armand010/PembMesin_Ganjil_2024/blob/main/Week10_Tugas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Identitas Kelompok
1. Armand Maulana Andika Putra (2241720090)
2. Bagus Arnovario Wibowo (2241720225)
3. Muhammad Harafsan Alhad (2241720059)
4. Yonatan Efrassetyo (2241720063)

Link Github: https://github.com/armand010/PembMesin_Ganjil_2024/blob/main/Week10_Tugas.ipynb



---

# Tugas
Prosedur pelatihan pada praktikum 2 merupakan prosedur sederhana, yang tidak memberi Anda banyak kendali. Model ini menggunakan "teacher-forcing" yang mencegah prediksi buruk diumpankan kembali ke model, sehingga model tidak pernah belajar untuk pulih dari kesalahan. Jadi, setelah Anda melihat cara menjalankan model secara manual, selanjutnya Anda akan mengimplementasikan custom loop pelatihan. Hal ini memberikan titik awal jika, misalnya, Anda ingin menerapkan pembelajaran kurikulum untuk membantu menstabilkan keluaran open-loop model. Bagian terpenting dari loop pelatihan khusus adalah fungsi langkah pelatihan.

Gunakan tf.GradientTape untuk men track nilai gradient. Anda dapat mempelajari lebih lanjut tentang pendekatan ini dengan membaca eager execution guide.

Prosedurnya adalah "

1. Jalankan Model dan hitung loss dengan tf.GradientTape.

2. Hitung update dan terapkan pada model dengan optimizer

In [ ]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

Kode diatas menerapkan train_step method sesuai dengan  Keras' train_step conventions. Ini opsional, tetapi memungkinkan Anda mengubah perilaku langkah pelatihan dan tetap menggunakan keras Model.compile and Model.fit methods.

In [ ]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
model.fit(dataset, epochs=1)

Atau jika ingin lebih mengetahui dalamnya, kita bisa membuat custom training loop sendiri:

In [ ]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Jalankan kode diatas dan sebutkan perbedaanya dengan praktikum 2?

---

# Jawaban

1. Kontrol dan Visibilitas Pelatihan:
  - Original (Praktikum 2):
    - Menggunakan Keras model.fit() standar
    - Proses pelatihan tersembunyi/abstrak
    - Kurang fleksibel untuk modifikasi
  - New (Tugas):
    - Memberikan kontrol yang lebih detail melalui:
    - Method train_step yang bisa dikustomisasi
    - Perhitungan gradient eksplisit menggunakan tf.GradientTape
    - Pelacakan metrik manual
    - Pelaporan progress yang lebih detail per batch

2. Pemulihan dari Kesalahan:
  - Original:
    - Menggunakan "teacher forcing" dimana model hanya melihat urutan yang benar selama pelatihan
    - Tidak belajar memulihkan diri dari kesalahan
  - New:
    - Bisa dimodifikasi agar model belajar dari kesalahannya
    - Lebih fleksibel dalam penanganan kesalahan prediksi

3. Struktur Implementasi:
  - Original:
```
model.compile(optimizer='adam', loss=loss)
history = model.fit(dataset, epochs=5, callbacks=[checkpoint_callback])
```
  - New:
    - Menggunakan konvensi Keras
```
class CustomTraining(MyModel):
    @tf.function
    def train_step(self, inputs):
        # Logika pelatihan kustom disini
```
    - Loop pelatihan sepenuhnya kustom
```
for epoch in range(EPOCHS):
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        # Pelacakan metrik dan pelaporan manual
```
4. Fleksibilitas dan Ekstensibilitas:
  - Original:
    - Terbatas pada prosedur pelatihan bawaan Keras
    - Sulit dimodifikasi
  - New:
    - Perhitungan loss yang dikustomisasi
    - Manipulasi gradient yang lebih detail
    - Pemantauan progress pelatihan yang lebih baik
    - Implementasi pembelajaran kurikulum
    -  penyimpanan checkpoint yang bisa disesuaikan
5. Pemantauan Kinerja:
  - Original:
    - Hanya pelaporan tingkat epoch dasar
  - New:
    - Pemantauan lebih detail dengan:
      - Pelaporan loss per batch
      - Pelacakan metrik kustom
      - Pelacakan waktu eksplisit per epoch
      - Penyimpanan checkpoint lebih sering (setiap 5 epoch)



Keunggulan implementasi yang baru:

1. Memudahkan debug masalah pelatihan
2. Bisa menerapkan teknik pelatihan yang lebih advanced
3. Memantau dan menyesuaikan proses pelatihan secara real-time
4. Memodifikasi cara model belajar dari kesalahannya

Dengan implementasi yang baru, kita mendapatkan kontrol penuh atas proses pelatihan dan bisa menyesuaikannya sesuai kebutuhan spesifik, berbeda dengan implementasi original yang lebih kaku dan terbatas pada fitur bawaan Keras.